In [ ]:
import trimesh
import open3d as o3d
import trimesh.exchange.stl

import numpy as np
import matplotlib.pyplot as plt

from common_functions import *

# Modify grain geometry from tomography output

In [ ]:
isolated_grains = trimesh.load_mesh('grain-geometries-CAD/box1_isolatedgrains-packed3.stl') 
#isolated_grains.apply_scale(1e-3)

In [ ]:
isolated_grains.show()

In [ ]:
# Convert Trimesh → Open3D
mesh_o3d = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(isolated_grains.vertices),
    triangles=o3d.utility.Vector3iVector(isolated_grains.faces)
)

# Compute normals (optional but useful for remeshing)
mesh_o3d.compute_vertex_normals()

# Apply Loop subdivision
mesh_remesh = mesh_o3d.subdivide_loop(number_of_iterations=2)

# Convert Open3D → Trimesh
vertices = np.asarray(mesh_remesh.vertices)
faces = np.asarray(mesh_remesh.triangles)
isolated_grains_remeshed = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)

centroid = isolated_grains_remeshed.centroid
# Translate the mesh so that the centroid is at (0, 0, 0)
isolated_grains_remeshed.apply_translation(-centroid)

# Visualize with Trimesh
isolated_grains_remeshed.show()

In [ ]:
# Get bounding box corners
min_corner, max_corner = isolated_grains_remeshed.bounds  # shape (2, 3)

# Compute size along each axis (X, Y, Z)
size = max_corner - min_corner

print(f"Bounding box min corner: {min_corner}")
print(f"Bounding box max corner: {max_corner}")
print(f"Size (X x Y x Z): {size}")

In [ ]:
isolated_grains_remeshed.apply_translation((0,0,-0.015))

In [ ]:
## determine appropiate grid spacing for Electric Field calculations

# Create mesh grid for exact sampling
WorldX, WorldY, WorldZ = 117.00044-10, 120.77438, 100.29045+15
print(117.00044-10, 120.77438, 100.29045)
stepsize = 10

x_array = np.arange(-WorldX/2, WorldX/2, stepsize)/1000
y_array = np.arange(-WorldY/2, WorldY/2, stepsize)/1000
z_array = np.arange(-WorldZ/2, WorldZ/2, stepsize)/1000

# Create 3D mesh grid
X, Y, Z = np.meshgrid(x_array, y_array, z_array, indexing='ij')

# Flatten the mesh grid to create sampling points
sampling_points = np.column_stack([X.ravel(), Y.ravel(), Z.ravel()])
print(len(sampling_points))
photoelectron_stopping_sites = trimesh.points.PointCloud(sampling_points, colors=[0, 0, 255, 255])

scene = plot_trimesh_edges_only(isolated_grains_remeshed, edge_color=[0, 0, 0, 128])
scene.add_geometry([photoelectron_stopping_sites])
scene.show()

In [ ]:
# Get ASCII STL string
ascii_stl_str = trimesh.exchange.stl.export_stl_ascii(mesh=isolated_grains_remeshed)

# Write it to a file
with open("../sphere-charging/geometry/isolated_grains_interpolated.stl", "w") as f:
    f.write(ascii_stl_str)

# PROBABLY NEED TO FIX THE SCALING 